Part 2 – Lambdas, Map, filter and reduce
=========================================

# Description
This part is designed to measure and assess your ability to:
- Design and use simple regular expressions.
- Create and use lambdas and higher order functions.

This part of the project builds upon the program you wrote for Part1 – Designing Classes Using
Python. We will use the zoo and all the classes of animals and birds we previously created. Below is
the organizational chart for the classes we created in the previous assignment.

We will provide additional functionality to the zoo class in this part and amend the existing classes.

# Instructions
Amend the classes to include the following functionality.
1. Modify the method to add animals or bids to the zoo.
    a. Using filter function, add functionality to ensure that only one object of each animal class can be added to the zoo. For example, zoo can have only one tiger. If we try to add more, print a message stating, animal already added.
2. Amend your looking method to use map and reduce function and create a single string to represent the zoo.
3. Add one method to look at all the canines in the zoo, use a filter function with lambdas for it.
4. Add a method to filter out tiger in the zoo and look at them. Use regex to achieve this functionality.
5. Hint: you will have to go over all the animals in the zoo.

In [1]:
# %%

from dataclasses import dataclass, field
from functools import reduce
import re

In [2]:
# %%

@dataclass
class Animal:
    """Defines an animal.

    Attributes:
        name: Name of the animal
        hands: Number of hands animal have
        legs: Number of legs animal have
    """

    name: str
    hands: int
    legs: int


@dataclass
class Feline(Animal):
    """Defines a Feline type animal.

    Attributes:
        family: Family of the feline. Defaults to "Cat".
        hands: Number of hands Feline have. Defaults to 0.
        legs: Number of legs Feline have. Defaults to 4.
    """

    family: str = field(init=False, default="Cat")
    hands: int = field(init=False, default=0)
    legs: int = field(init=False, default=4)


@dataclass
class Canine(Animal):
    """Defines a Canine type animal.

    Attributes:
        family: Family of the Canine. Defaults to "Dog".
        hands: Number of hands Canine have. Defaults to 0.
        legs: Number of legs Canine have. Defaults to 4.
    """

    family: str = field(init=False, default="Dog")
    hands: int = field(init=False, default=0)
    legs: int = field(init=False, default=4)


@dataclass
class Tiger(Feline):
    """Define a tiger type feline.

    Attributes:
        predator_type: Defines the type of predator tiger is. Defaults to "Lethal".
        sound: Sound made by tiger. Defaults to "Roaaaaarrrr!!".
    """

    predator_type: str = field(init=False, default="Lethal")
    sound: str = field(init=False, default="Roaaaaarrrr!!")


@dataclass
class WildCat(Feline):
    """Define a wild cat type feline.

    Attributes:
        climb: Depicts climbing nature of wild cat. Defaults to "Climbing Tree!".
    """

    climb: str = field(init=False, default="Climbing Tree!")


@dataclass
class Wolf(Canine):
    """Define a wolf type canine.

    Attributes:
        is_leader: Defines if the wolf is leader of the pack or not.
        hunt: Defines the hunting style of wolf. Defaults to "the pack for hunt.".
    """

    is_leader: bool
    hunt: str = field(init=False, default="the pack for hunt.")

    def __post_init__(self):  # noqa: D105
        # If the wolf is the leader, update the hunt string to reflect that.
        self.hunt = f"Leading {self.hunt}" if self.is_leader else f"Following {self.hunt}"

In [3]:
# %%

@dataclass
class Bird:
    """Defines a bird.

    Attributes:
        name: Name of the bird.
        legs: Number of legs bird have.
        wings: Number of wings bird have.
    """

    name: str
    legs: int
    wings: int


@dataclass
class FlightBird(Bird):
    """Defines birds that can fly.

    Attributes:
        legs: Number of legs FlightBird have. Defaults to 2.
        wings: Number of wings FlightBird have. Defaults to 2.
        fly: Depicts flying nature of FlightBird. Defaults to "Flying!!".
        hunt: Depicts hunting style of FlightBird. Defaults to "Hunting for food!!".
    """

    legs: int = field(init=False, default=2)
    wings: int = field(init=False, default=2)
    fly: str = field(init=False, default="Flying!!")
    hunt: str = field(init=False, default="Hunting for food!!")


@dataclass
class Eagle(FlightBird):
    """Defines eagle flight bird.

    Attributes:
        fly: Depicts flying nature of eagle. Defaults to "Flying extremely high!!".
    """

    fly: str = field(init=False, default="Flying extremely high!!")

    def __post_init__(self):  # noqa: D105
        # Update the hunt string to include the eagle's hunting style.
        self.hunt = f"Spotting prey from high up in the sky. {self.hunt}"

In [4]:
# %%

from typing import Union


class Zoo:
    """Define a zoo of configurable capacity."""

    def __init__(self, bird_capacity: int = 1, animal_capacity: int = 2):
        """Initialize Zoo class.

        Args:
            bird_capacity: Number of birds the zoo can hold
            animal_capacity: Number of animals the zoo can hold

        """
        # Initialize the animals list, bird capacity, and animal capacity.
        self.__animals: list[Union[Animal, Bird]] = []
        self.__bird_capacity = bird_capacity
        self.__animal_capacity = animal_capacity

    @property
    def capacity(self) -> tuple[int, int]:
        """Get the capacity of the zoo.

        Returns:
            A tuple containing the bird capacity and animal capacity.
        """
        return (self.__bird_capacity, self.__animal_capacity)

    def add(self, animal: Union[Animal, Bird]):
        """Add animal to the zoo.

        Args:
            animal: Animal to be added in the zoo.

        Raises:
            IndexError: If zoo is full or if an animal of the same type is already present.
            TypeError: If incorrect type of animal is provided.
        """
        # Check if the zoo is full.
        if len(self.__animals) >= (self.__animal_capacity + self.__bird_capacity):
            raise IndexError("Zoo is full!")
        # Check if an animal of the same type is already present.
        elif len(list(filter(lambda x: isinstance(x, type(animal)), self.__animals))) > 0:
            raise IndexError(f"Zoo can only have 1 Animal/Bird of type {type(animal)}")
        # Check if the animal is a bird and if there is space for birds.
        elif (
            (
                len(
                    [curr_animal for curr_animal in self.__animals if isinstance(curr_animal, Bird)]
                )
                < self.__bird_capacity
            )
            and isinstance(animal, Bird)
        ) or (
            # Check if the animal is an animal and if there is space for animals.
            len([curr_animal for curr_animal in self.__animals if isinstance(curr_animal, Animal)])
            < self.__animal_capacity
            and isinstance(animal, Animal)
        ):
            # Add the animal to the zoo.
            self.__animals.append(animal)
        else:
            # Raise an error if the animal type is invalid.
            raise TypeError(f"Invalid type {type(animal)}")

    def look(self, animals=None):
        """Prints all the animals in the zoo.

        Args:
            animals: List of animals to look at. Defaults to all animals in the zoo.
        """
        # If no animals are provided, use all animals in the zoo.
        animals = self.__animals if not animals else animals
        # Use map to get the string representation of each animal.
        # Use reduce to concatenate the string representations with newlines.
        representations = reduce(lambda a, b: a + "\n" + b, map(lambda x: x.__repr__(), animals))
        # Print the representations.
        print(representations)

    def look_canines(self):
        """Prints all the canines in the zoo."""
        # Use filter to get all canines in the zoo.
        canines = list(filter(lambda x: isinstance(x, Canine), self.__animals))
        # Call the look method with the canines.
        self.look(animals=canines)

    def look_tiger(self):
        """Prints all the tigers in the zoo."""
        # Use filter and regex to get all tigers in the zoo.
        tigers = list(filter(lambda x: re.search("Tiger", type(x).__name__), self.__animals))
        # Call the look method with the tigers.
        self.look(animals=tigers)

In [6]:
# %%

eagle = Eagle("Jatayu")
tiger = Tiger("Simba")
jaguar = WildCat("Bagheera")
wolf = Wolf("Nymeria", True)

zoo = Zoo()
zoo.add(eagle)
zoo.add(tiger)
# zoo.add(jaguar)

# Raise error if trying to add more.
zoo.add(wolf)
# Raise error if trying another animal of existing type.
# zoo.add(tiger)

zoo.look()

Eagle(name='Jatayu', legs=2, wings=2, fly='Flying extremely high!!', hunt='Spotting prey from high up in the sky. Hunting for food!!')
Tiger(name='Simba', hands=0, legs=4, family='Cat', predator_type='Lethal', sound='Roaaaaarrrr!!')
Wolf(name='Nymeria', hands=0, legs=4, family='Dog', is_leader=True, hunt='Leading the pack for hunt.')


In [7]:
# %%

zoo.look_canines()

Wolf(name='Nymeria', hands=0, legs=4, family='Dog', is_leader=True, hunt='Leading the pack for hunt.')


In [8]:
# %%

zoo.look_tiger()

Tiger(name='Simba', hands=0, legs=4, family='Cat', predator_type='Lethal', sound='Roaaaaarrrr!!')
